In [ ]:
!pip install boto3 PyPDF2==1.27.8

# **Application of tempfile**

In [ ]:
import tempfile
from time import sleep
import boto3
from PyPDF2 import PdfFileReader

In [ ]:
with tempfile.NamedTemporaryFile(delete=True) as tmp_file:
  local_filename = tmp_file.name
  print("The local file created in tmp directory is ",local_filename)
  sleep(60)

In [ ]:
bucket_name=''
s3_file_name=''
s3_client=boto3.client('s3',aws_access_key_id='',aws_secret_access_key='',region_name='us-east-1')
with tempfile.NamedTemporaryFile(delete=True) as tmp_file:
  local_filename = tmp_file.name
  print("The local file created in tmp directory is ",local_filename)
  s3_client.download_file(
        Bucket=bucket_name,
        Key=s3_file_name,
        Filename=local_filename
        )
  pdf_object=PdfFileReader(local_filename)
  no_of_pages=len(pdf_object.pages)
  print("No. of pages in this pdf file is: ",no_of_pages)
  sleep(40)

# **Usage in Lambda Function**

In [ ]:
import json
import tempfile
import boto3
from PyPDF2 import PdfFileReader
s3_client=boto3.client('s3',region_name='us-east-1')

def lambda_handler(event, context):
    # TODO implement
    bucket_name = event["Records"][0]["s3"]["bucket"]["name"]
    s3_file_name = event["Records"][0]["s3"]["object"]["key"]
    print("Bucket : ",bucket_name)
    print("File Name : ",s3_file_name)
    with tempfile.NamedTemporaryFile(delete=True) as tmp_file:
        local_filename = tmp_file.name
        print("The local file : ",local_filename)
        s3_client.download_file(
        Bucket=bucket_name,
        Key=s3_file_name,
        Filename=local_filename
        )
        pdf_object=PdfFileReader(local_filename)
        no_of_pages=len(pdf_object.pages)
        print("No. of pages in this pdf file is: ",no_of_pages)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

# **Application of tempdir**

In [ ]:
!pip install img2pdf

In [ ]:
import img2pdf
import boto3
import tempfile
import os
from time import sleep
bucket_name=''
s3_file_name_with_dir=''
only_image_file_name=s3_file_name_with_dir.split('/')[-1]
s3_client=boto3.client('s3',aws_access_key_id='',aws_secret_access_key='',region_name='us-east-1')

tmpdir = tempfile.mkdtemp()
image_file = os.path.join(tmpdir, only_image_file_name)
print("The image file to be written at: ",image_file)
s3_client.download_file(
        Bucket=bucket_name,
        Key=s3_file_name_with_dir,
        Filename=image_file
        )

pdf_file_name=image_file.replace('.JPG','.pdf')
print("The pdf file to be written at: ",pdf_file_name)
with open(pdf_file_name,"wb") as f:
  f.write(img2pdf.convert(image_file))

s3_client.upload_file(pdf_file_name, bucket_name, s3_file_name_with_dir.replace('.JPG','.pdf'))
sleep(60)
os.remove(image_file)
os.remove(pdf_file_name)
os.rmdir(tmpdir)

# **Usage in Lambda Function**

In [ ]:
import json
import img2pdf
import boto3
import tempfile
import os
from time import sleep
s3_client=boto3.client('s3',region_name='us-east-1')

def lambda_handler(event, context):
    # TODO implement
    bucket_name = event["Records"][0]["s3"]["bucket"]["name"]
    s3_file_name = event["Records"][0]["s3"]["object"]["key"]
    print("Bucket : ",bucket_name)
    print("Landed s3 file : ",s3_file_name)
    only_image_file_name=s3_file_name.split('/')[-1]
    tmpdir = tempfile.mkdtemp()
    image_file = os.path.join(tmpdir, only_image_file_name)
    print("The image file to be written at: ",image_file)
    s3_client.download_file(
            Bucket=bucket_name,
            Key=s3_file_name,
            Filename=image_file
            )

    pdf_file_name=image_file.replace('.JPG','.pdf')
    print("The pdf file to be written at: ",pdf_file_name)
    with open(pdf_file_name,"wb") as f:
      f.write(img2pdf.convert(image_file))

    s3_client.upload_file(pdf_file_name, bucket_name, s3_file_name.replace('.JPG','.pdf'))
    sleep(30)
    os.remove(image_file)
    os.remove(pdf_file_name)
    os.rmdir(tmpdir)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }